# ICPO4 DECISION TREES

## HEMA SAI ARI (U59528014)

# Lawn Mower ownership prediction using SVM models

We will predict if the person is owner of lawn mower or not.

## 1. Setup

In [1]:
# Common imports
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import GridSearchCV


np.random.seed(1)

# 2. Load the data

In [2]:
# Uncomment the following snippet of code to debug problems with finding the .csv file path
# This snippet of code will exit the program and print the current working directory.
#import os
#print(os.getcwd())

In [3]:
X_train = pd.read_csv("C:/Users/arihe/Box/SEM-2/DATA SCIENCE PRORAMMING/SIR PYTHON FIES/class-4(Decision Trees)/airbnb_train_X_price_gte_150.csv")
X_test = pd.read_csv("C:/Users/arihe/Box/SEM-2/DATA SCIENCE PRORAMMING/SIR PYTHON FIES/class-4(Decision Trees)/airbnb_test_X_price_gte_150.csv")
y_train = pd.read_csv("C:/Users/arihe/Box/SEM-2/DATA SCIENCE PRORAMMING/SIR PYTHON FIES/class-4(Decision Trees)/airbnb_train_y_price_gte_150.csv")
y_test = pd.read_csv("C:/Users/arihe/Box/SEM-2/DATA SCIENCE PRORAMMING/SIR PYTHON FIES/class-4(Decision Trees)/airbnb_test_y_price_gte_150.csv")

## 3.0 Model the data using the SVM models and Decision Tree using Gridsearch

First, let's create a dataframe to load the model performance metrics into.



In [4]:
performance_svm = pd.DataFrame({"model": [], "Accuracy": [], "Precision": [], "Recall": [], "F1": []})


### 3.1 Fit a SVM classification model using polynomial kernal¶

In [5]:
svm_poly_model = SVC(kernel="poly", degree=3, coef0=1, C=10)
_ = svm_poly_model.fit(X_train, np.ravel(y_train))

In [6]:
model_preds = svm_poly_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance_svm = pd.concat([performance_svm, pd.DataFrame({'model':"poly svm", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

##  Summary for SVM Models

In [7]:
performance_svm

,model,Accuracy,Precision,Recall,F1
0,poly svm,0.867854,0.855839,0.883239,0.869323


### 3.2 Fit a DTree classification model using Grid Search 

In [8]:
score_measure = "precision"
kfolds = 5

param_grid = {
    'min_samples_split': [1,10,100,200],  
    'min_samples_leaf': [1,5,20,50],
    'min_impurity_decrease': [0.0001, 0.0005, 0.0020, 0.0050],
    'max_leaf_nodes': [10,25,100,200], 
    'max_depth': [5,10,30], 
    'criterion': ['entropy', 'gini'],
}

dtree = DecisionTreeClassifier()
grid_search = GridSearchCV(estimator = dtree, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestRecallTree = grid_search.best_estimator_

Fitting 5 folds for each of 1536 candidates, totalling 7680 fits
The best precision score is 0.8581623158208013
... with parameters: {'criterion': 'entropy', 'max_depth': 10, 'max_leaf_nodes': 25, 'min_impurity_decrease': 0.0001, 'min_samples_leaf': 5, 'min_samples_split': 10}


C:\Users\arihe\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
1920 fits failed out of a total of 7680.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1920 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\arihe\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\arihe\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 969, in fit
    super().fit(
  File "C:\Users\arihe\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 265, in fit
    check_scalar(
  File "C:\Users\arihe\anaconda3\lib\site-packages\sklearn\utils\validation.py"

In [9]:
c_matrix = confusion_matrix(y_test,  grid_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
print(f"Accuracy={(TP+TN)/(TP+TN+FP+FN):.7f} Precision={TP/(TP+FP):.7f} Recall={TP/(TP+FN):.7f} F1={2*TP/(2*TP+FP+FN):.7f}")

Accuracy=0.8388004 Precision=0.8655804 Recall=0.8003766 F1=0.8317025


## Conclusion

We can observe that the precision of the model on the data is 0.8558

We can observe that the precision of the model on the data is 0.865 but the best precision score in the given range of parameters is 0.8581

The precision of both SVM with poly kernal and the Decision tree using gridsearch have almost the same precision(Decision Tree has a little bit more value than the SVM).Which indicates that both the models are predicting the data at same precision. So when a question arises about the best model we will choose the **Decision Tree** because it is simple and easy to interpret and it can be pruned further to get much stable predictions. And also the decision tree have a more precsion value than the SVM in on the test data. The decison tree's best precision score on the train data is 0.858 which is almost equal to the precision value of the SVM.which indicates that the model isn't performing good on the train data but its performing good on the test data.